In [1]:
import pandas as pd

In [4]:
df2 = pd.read_csv("new_merged_features_IC50_g12d.csv")
df2

<>:1: SyntaxWarning: invalid escape sequence '\D'
<>:1: SyntaxWarning: invalid escape sequence '\D'
C:\Users\Gebruiker\AppData\Local\Temp\ipykernel_28520\1216648286.py:1: SyntaxWarning: invalid escape sequence '\D'
  df2 = pd.read_csv("C:\\Gebruiker\Desktop\pycharm\new_merged_features_IC50_g12d.csv")
C:\Users\Gebruiker\AppData\Local\Temp\ipykernel_28520\1216648286.py:1: SyntaxWarning: invalid escape sequence '\D'
  df2 = pd.read_csv("C:\\Gebruiker\Desktop\pycharm\new_merged_features_IC50_g12d.csv")


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Gebruiker\\Desktop\\pycharm\new_merged_features_IC50_g12d.csv'

In [3]:
import numpy as np
import pandas as pd
def pIC50(input):
    pIC50 = []

    input["IC50 (nM)"] = pd.to_numeric(input["IC50 (nM)"],errors='coerce')

    for i in input["IC50 (nM)"]:
        molar = i*(10**-9) # Converts nM to M
        pIC50.append(-np.log10(molar))

    input['pIC50'] = pIC50
    x = input["pIC50"]

    return x

In [4]:
filename = "merged_features_IC50_g12d.csv"
#df = pd.read_csv("fda_original.csv",sep=";")
df2 = pd.read_csv(filename)
df2.dropna(subset=['IC50 (nM)'], inplace=True)
df2['IC50 (nM)'] = pIC50(df2)
df2['IC50 (nM)'] = df2['pIC50']  # Replace 'SV' column with pIC50 values

# Drop 'pIC50' column (optional) as it's now redundant
df2.drop(columns=['pIC50'], inplace=True)

df2

,ChEMBL ID,Smiles,FC,N,H,Cl,Br,I,P,F,...,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea,IC50 (nM)
0,1340697,Oc1cc(-c2nc3OC[C@@H]4[C@@H]5CC[C@H](CN4c4nc(OC...,0,348,33,0,0,0,0,2,...,0,0,1,0,0,0,0,0,0,9.69897
1,1340710,Oc1cc(-c2nc3OC[C@@H]4[C@@H]5CC[C@H](CN4c4nc(OC...,0,362,35,0,0,0,0,2,...,0,0,1,0,0,0,0,0,0,9.69897
2,1340721,Oc1cc(-c2nc3CC[C@@H]4[C@@H]5CC[C@H](CN4c4nc(OC...,0,440,35,0,0,0,0,11,...,0,0,1,0,0,0,0,0,0,9.69897
3,1340735,C[Si]1(C)CCCN(CC2(COc3nc4N5C[C@H]6CC[C@H](N6)[...,0,368,44,0,0,0,0,2,...,0,0,1,0,0,0,0,0,0,9.69897
4,1340738,C[C@H]1C[C@@H]2[C@@H]3CC[C@H](CN2c2nc(OC[C@@]4...,0,332,34,0,0,0,0,4,...,0,0,0,0,0,0,0,0,0,9.69897
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3519,1209825,Oc1cc(-c2ncc3c(nc(OC[C@@]45CCCN4C[C@H](F)C5)nc...,0,310,31,1,0,0,0,2,...,0,0,0,0,0,0,0,0,0,9.69897
3520,1340667,Oc1cc(-c2nc3OC[C@@H]4[C@@H]5CC[C@H](CN4c4nc(OC...,0,328,31,0,0,0,0,3,...,0,0,1,0,0,0,0,0,0,9.69897
3521,1340691,Oc1cc(-c2nc3OC[C@@H]4[C@@H]5CC[C@H](CN4c4nc(OC...,0,336,30,0,0,0,0,4,...,0,0,1,0,0,0,0,0,0,9.69897
3522,1340693,Oc1cc(-c2nc3OC[C@@H]4[C@@H]5CC[C@H](CN4c4nc(OC...,0,344,35,0,0,0,0,3,...,0,0,1,0,0,0,0,0,0,9.69897


In [5]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import warnings
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

# Load training descriptors data
#df1 = df1.loc[:, ~df1.columns.str.contains('Unnamed')]
df2.dropna(subset=['IC50 (nM)'], inplace=True)
X = df2.drop(columns=['IC50 (nM)','Smiles', 'ChEMBL ID'])
y = df2['IC50 (nM)']

# Initialize model and pipeline
model = RandomForestRegressor(max_depth=20, min_samples_leaf=10)
pipe = Pipeline([('scaler', StandardScaler()), ('model', model)])

# Load new descriptors data
new_descriptors = pd.read_csv('FDA_Hyb_Features.csv')
new_descriptors.dropna(inplace=True)

# Extract ChEMBL IDs and SMILES
chembl_id_column = new_descriptors['ChEMBL ID']
smiles_column = new_descriptors['Smiles']
new_descriptors.drop(columns=['ChEMBL ID', 'Smiles'], inplace=True)

# Ensure new_descriptors has the same columns as X
missing_cols = set(X.columns) - set(new_descriptors.columns)  # Columns missing in new data
extra_cols = set(new_descriptors.columns) - set(X.columns)  # Extra columns in new data

# Add missing columns with zero values
for col in missing_cols:
    new_descriptors[col] = 0

# Drop extra columns
new_descriptors = new_descriptors[X.columns]  # Reorder and drop extras

# Now X and new_descriptors have identical columns

# Initialize a DataFrame to store results
loop_results_df = pd.DataFrame(columns=['ChEMBL ID', 'Smiles', 'Predicted Value'])

# Number of iterations
loop_n = 50

# Dictionary to store predicted values for each ChEMBL ID
predicted_values = {chembl_id: [] for chembl_id in chembl_id_column.unique()}

# Run the loop to fit the model and predict the values
for i in range(loop_n):
    pipe.fit(X, y)

    loop_predicted_values = pipe.predict(new_descriptors)

    loop_df = pd.DataFrame({
        'ChEMBL ID': chembl_id_column,
        'Smiles': smiles_column,
        'Predicted Value': loop_predicted_values
    })
    loop_df.sort_values(by='Predicted Value', ascending=False, inplace=True)

    with warnings.catch_warnings():
        warnings.filterwarnings("ignore", category=FutureWarning)
        loop_results_df = pd.concat([loop_results_df, loop_df.head(10)])

    # Store the predicted values for each ChEMBL ID
    for chembl_id, predicted_value in zip(chembl_id_column, loop_predicted_values):
        if chembl_id in predicted_values:
            predicted_values[chembl_id].append(predicted_value)

# Calculate frequency of each ChEMBL ID in the top 10
value_counts = loop_results_df['ChEMBL ID'].value_counts(normalize=True) * 10
#print(value_counts)

# Get the top 13 ChEMBL IDs
top_10_chembl_ids = value_counts.head(10).index

In [6]:
mean_values = {chembl_id: np.mean(predicted_values[chembl_id]) for chembl_id in top_10_chembl_ids}
std_dev_values = {chembl_id: np.std(predicted_values[chembl_id]) for chembl_id in top_10_chembl_ids}

summary_df = pd.DataFrame({
    'ChEMBL ID': top_10_chembl_ids,
    'Frequency': [value_counts[chembl_id] for chembl_id in top_10_chembl_ids],
    'Avg Predicted Value': [mean_values[chembl_id] for chembl_id in top_10_chembl_ids]
})

summary_df.head(10)
summary_df.to_csv("RF_molecules_Newfeatures_G12D.csv", index=False)

,ChEMBL ID,Frequency,Avg Predicted Value
0,CHEMBL1200503,0.333333,7.843862
1,CHEMBL1200515,0.333333,7.276169
2,CHEMBL493,0.333333,7.803385
3,CHEMBL5095050,0.333333,7.031920
4,CHEMBL4298128,0.333333,7.025578
5,CHEMBL5315124,0.333333,7.359843
6,CHEMBL1201244,0.333333,7.049973
7,CHEMBL3989850,0.333333,7.098618
8,CHEMBL3545309,0.333333,7.281790
9,CHEMBL1410,0.333333,7.111971
